This script is aimed at extracting the source from the gtd case details and googling the source used

In [ ]:
import pandas as pd
import ast
import re
from dateutil import parser
from IPython.display import display

df = pd.read_csv('2_GTD_case_details_2015_2017.csv')
display(df.columns)
df = df[['GTD ID:','1_src_0', '1_src_1','1_src_2']]
df.rename(columns={'GTD ID:':'GTD ID'}, inplace=True)

In [ ]:
#return headline,news source,date, year
def strip_src(src):
    if type(src)!=float:
        b = ast.literal_eval(src)
        headline = re.findall(r'"([^"]*)"',b[0])[0]

        lst = b[0].replace('\"','')
        lst = re.split(",",lst)
        dates = lst[-2:]
        dates =','.join(dates).strip()
        source = str(lst[-3]).strip()
        df = pd.DataFrame([headline, source, dates]).transpose()
        df.columns=['headline','source','date']
    else:
        df = pd.DataFrame(columns=['headline','source','date'])
    return df

lst = ['1_src_0', '1_src_1', '1_src_2']

s_df = pd.DataFrame()

def unstack(df,a):
    scrape_df = pd.DataFrame()
    for i in range(len(df)):
        gtd_id = pd.DataFrame([df['GTD ID'].iloc[i]])
        gtd_id.columns=['GTD ID']
        sdf = strip_src(df[a].iloc[i])
        sdf = pd.concat([gtd_id,sdf],axis=1)
        scrape_df = pd.concat([scrape_df,sdf])
    return scrape_df

for i in lst:
    sdf=unstack(df,i)
    s_df=pd.concat([s_df,sdf])
s_df


In [ ]:
#cleaning the dataset
scrape_df = s_df.drop_duplicates().dropna(subset=['headline']).reset_index(drop=True)
scrape_df['headline'] = scrape_df['headline'].apply(lambda x: x[:-1])
display(scrape_df)
scrape_df.to_csv('0.1_google_search_link.csv')

In this portion, we will be using the headlines, source and the date to search and scrape Google links online.

In [1]:
import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys
from random import randint
import time
import pandas as pd

#unusable sources
scrape_df= pd.read_csv('0.1_google_search_link.csv').drop('Unnamed: 0',axis=1)
display(scrape_df.head())
unusable_sources = ['OSC Summary', 'Pattani Southern Border Provinces News Database', 
                   'Southern Border Provinces News Database','Summary','US Army Asian Studies Detachment',
                   'US Embassy Public Affairs Section'
                    ]
scrape_df = scrape_df[~scrape_df.source.isin(unusable_sources)]
scrape_df.sort_values('source')

def construct_search_terms(sdf):
    search_terms = str('"' + sdf['headline'] + '" ' + sdf['source'] + ' ' +sdf['date'])
    return search_terms

display(construct_search_terms(scrape_df.iloc[0]))

print('number of articles to search for: ' + str(len(scrape_df)))

main_df = pd.DataFrame()

#scrape the links in the first pg
def scrape_first_pg_links(search_terms,driver):
    # navigate to the application home page
    driver.get("http://www.google.com.sg")

    # get the search textbox
    search_field = driver.find_element_by_css_selector(".gLFyf")
    search_field.clear()

    # enter search keyword and submit
    search_field.send_keys(search_terms)
    rand = randint(8, 15)
    driver.implicitly_wait(rand)
    search_field.submit()

    links = []
    g_links = driver.find_elements_by_css_selector("div.g") 
    for i in g_links:
        link = i.find_element_by_tag_name("a")
        href = link.get_attribute("href")
        links.append(href)
        
    #if encounter a captcha
    if driver.find_elements_by_css_selector("#recaptcha"): 
        raise Exception('An Exception has occured: recaptcha encountered')     
    
    return links



,GTD ID,headline,source,date
0,201712310032,Security tightened in Cotabato following IED d...,Tempo,"January 4, 2018."
1,201712310030,Maguindanao clashes trap tribe members,Philippines Daily Inquirer,"January 3, 2018."
2,201712310016,"Cop, 2 others killed in bomb blasts in Mindanao",Philippines Daily Inquirer,"January 1, 2018."
3,201712310011,Cops hunt North Cotabato bombers,Philippines News Agency,"January 2, 2018."
4,201712310010,3 slain in Maguindanao roadside bombings,Philippine Star,"January 1, 2018."


'"Security tightened in Cotabato following IED discovery" Tempo January 4, 2018.'

number of articles to search for: 4801


There are 2 pieces of code here. One for manual fetching and the other is for auto fetching. Manual fetching is preferred as it enables us to be faster by playing captcha. Future iterations will see me make this into a class for easier mgmt

In [3]:
import winsound

def beep():
    duration = 1000  # milliseconds
    freq = 440  # Hz
    winsound.Beep(freq, duration)
    
beep()

In [27]:
import winsound

def beep():
    duration = 1000  # milliseconds
    freq = 440  # Hz
    winsound.Beep(freq, duration)

#auto
def auto_fetch(n1,n2,outputcsv,df):
    timelapse = 110 #1 hour 50 min timelapse 
    for i in range(n1,n2):
        print(i)
        while True:
            try:
                driver = webdriver.Firefox()
                driver.implicitly_wait(10)
                driver.maximize_window()
                
                st=(construct_search_terms(df.iloc[i]))
                print(st)
                link = pd.Series(scrape_first_pg_links(st,driver)) #returns a str
                l_df = pd.DataFrame([df.iloc[i].append(link)])
                l_df.to_csv(outputcsv,mode='a',header=False)
                driver.quit() 
                
            except Exception as e:
                print(e)
                driver.quit()      

                print('waiting... trying again in '+str(timelapse))
                for i in range(timelapse):
                    time.sleep(60) 
                    print('waiting... trying again in '+str(timelapse-i))
                continue
                
            else:
                break



def man_fetch(n1,n2,outputcsv,df):
    driver = webdriver.Firefox()
    #driver.implicitly_wait(10)
    driver.maximize_window()

    for i in range(n1,n2):
        print(i)
        while True:
            try:
                st=(construct_search_terms(df.iloc[i]))
                print(st)
                link = pd.Series(scrape_first_pg_links(st,driver)) #returns a str
                l_df = pd.DataFrame([df.iloc[i].append(link)])
                l_df.to_csv(outputcsv,mode='a',header=False)
            except Exception as e:
                print(e)
                beep()
                timelapse = 3 
                print('solve captcha now!',end='\r')
                time.sleep(timelapse*60)
                continue
            else:
                break
    driver.quit()


In [28]:
s_df = scrape_df[['headline','source','date']]
s_df = s_df.drop_duplicates()
s_df

,headline,source,date
0,Security tightened in Cotabato following IED d...,Tempo,"January 4, 2018."
1,Maguindanao clashes trap tribe members,Philippines Daily Inquirer,"January 3, 2018."
2,"Cop, 2 others killed in bomb blasts in Mindanao",Philippines Daily Inquirer,"January 1, 2018."
3,Cops hunt North Cotabato bombers,Philippines News Agency,"January 2, 2018."
4,3 slain in Maguindanao roadside bombings,Philippine Star,"January 1, 2018."
6,NPA abducts police officer in Cotabato,Manila Times (Philippines),"December 30, 2017."
7,"Cop slain, 5 colleagues wounded in BIFF attack...",Manila Bulletin,"January 1, 2018."
8,AFP troops deter harassment of communities by ...,Manila Bulletin,"December 30, 2017."
9,Montevista municipal councilor shot dead,Sun Star Network,"December 28, 2017."
10,"BIFF gunmen kill tribal chief, abduct 2 in Mag...",Manila Times,"December 28, 2017."


In [30]:
#max is 3345
auto_fetch(1318,1500,'0.1_1_google_link.csv',s_df)

1318
"PM to visit Thai Far South" Organization of Asia-Pacific News Agencies October 13, 2015.
1319
"Thailand: Bombs injure 4 policemen in Narathiwat" Bangkok Post Online October 12, 2015.
1320
"Bomb attempt in Cotabato City foiled" Philippines News Agency October 12, 2015.
1321
"Military condemns killing of mayor, son" EDGE Davao October 20, 2015.
1322
"Rogue MILF fighters eyed in Zamboanga Sibugay mayor's slay" Philippine Star October 20, 2015.
1323
"Antipolo City Council aspirant shot" Manila Bulletin October 12, 2015.
1324
"Village exec killed in gun attack" Philippine Star October 10, 2015.
1325
"10th ID troops pursuing NPA band who staged series of IED attacks in Compostela Valley" Philippines News Agency October 12, 2015.
1326
"'Al-Khobar' blamed for bombing of NGCP towers in North Cotabato" Philippine Daily Inquirer October 11, 2015.
1327
"Bombs toppled power towers in S. Philippines" Xinhua General News Service October 10, 2015.
1328
"Thailand: Three killed in two separate att

waiting... trying again in 105
waiting... trying again in 104
waiting... trying again in 103
waiting... trying again in 102
waiting... trying again in 101
waiting... trying again in 100
waiting... trying again in 99
waiting... trying again in 98
waiting... trying again in 97
waiting... trying again in 96
waiting... trying again in 95
waiting... trying again in 94
waiting... trying again in 93
waiting... trying again in 92
waiting... trying again in 91
waiting... trying again in 90
waiting... trying again in 89
waiting... trying again in 88
waiting... trying again in 87
waiting... trying again in 86
waiting... trying again in 85
waiting... trying again in 84
waiting... trying again in 83
waiting... trying again in 82
waiting... trying again in 81
waiting... trying again in 80
waiting... trying again in 79
waiting... trying again in 78
waiting... trying again in 77
waiting... trying again in 76
waiting... trying again in 75
waiting... trying again in 74
waiting... trying again in 73
wait

1418
"Vice mayor's husband shot in Bukidnon" Tempo August 17, 2015.
1419
"NPA burns truck, kills civilian" Sun Star Network August 17, 2015.
1420
"DPWH engineer killed in Pagadian City" Manila Times August 14, 2015.
1421
"Philippines: 3 injured as Abu Sayyaf attacks army" Muslim News August 15, 2015.
1422
"Village chief, escort slain in Pikit attack" Tempo August 15, 2015.
1423
"3 killed in separate attacks in Cotabato" Philippine Star August 14, 2015.
1424
"Priest, businessman hurt in Ilocos gun attack" Journal Online August 12, 2015.
1425
"Unidentified militants kill a former CAFGU militiaman in the Philippines' Negros Oriental" Jane's Terrorism Watch Report August 13, 2015.
1426
"NPA rebels gun down village councilor in Camarines Sur" HT Syndication August 8, 2015.
1427
"2 farmers hurt in Kidapawan strafing incident" Manila Bulletin August 8, 2015.
1428
"Thailand: Bomb kills marine, injures 3 in Narathiwat" The Nation Online August 6, 2015.
1429
"Philippines: Military on the lookout

waiting... trying again in 100
waiting... trying again in 99
waiting... trying again in 98
waiting... trying again in 97
waiting... trying again in 96
waiting... trying again in 95
waiting... trying again in 94
waiting... trying again in 93
waiting... trying again in 92
waiting... trying again in 91
waiting... trying again in 90
waiting... trying again in 89
waiting... trying again in 88
waiting... trying again in 87
waiting... trying again in 86
waiting... trying again in 85
waiting... trying again in 84
waiting... trying again in 83
waiting... trying again in 82
waiting... trying again in 81
waiting... trying again in 80
waiting... trying again in 79
waiting... trying again in 78
waiting... trying again in 77
waiting... trying again in 76
waiting... trying again in 75
waiting... trying again in 74
waiting... trying again in 73
waiting... trying again in 72
waiting... trying again in 71
waiting... trying again in 70
waiting... trying again in 69
waiting... trying again in 68
waiting..

In [16]:
man_fetch(234,3345,'0.1_1_google_link.csv',s_df)

234
"TWO POLICEMEN, WOMAN INJURED IN MOTORCYCLE BOMB ATTACK" Malaysia General News July 15, 2017.
235
"Sayyaf men kidnap 6 loggers, hunter" Philippine Star July 22, 2017.
236
"Reds torch Quezon construction firm equipment, bomb PAF truck" Journal Online July 16, 2017.
237
"Philippines: 6 hurt in landmine attack" Philstar.com July 15, 2017.
238
"2 kids hurt in Maguindanao grenade attack" Philippine Star July 14, 2017.
239
"Firefight, land mine blasts rock Masbate town" Philippines Daily Inquirer July 12, 2017.
240
"NPA abducts cop in DavOr" Minda News July 13, 2017.
241
"Philippines: Militiaman killed by suspected extremists" Philstar.com July 11, 2017.
242
"Thailand: Three-year-old boy escapes shooting in Pattani, but father seriously injured" Thai PBS Online July 10, 2017.
243
"NPA owns killing of NorthCot widow accused of feeding vital info to military" Minda News July 13, 2017.
244
"NPA rebels torch construction equipment" Philippine Star July 10, 2017.
245
"NPA behind RPA-ABB leade

An Exception has occured: recaptcha encountered


KeyboardInterrupt: 